In [1]:
import NN
import NNBij
import h5py
import json
import time
import pyccl
import scipy
import numpy
import random
from itertools import product
from matplotlib import pyplot
from matplotlib.gridspec import GridSpec

In [2]:
def C_function(coefficients, phi_grid1, phi_grid2, grid_size):
    outer_product = numpy.outer(phi_grid1, phi_grid2)

    functions = numpy.sum(coefficients * outer_product[:, :, None], axis=(0, 1))
    
    return functions

In [3]:
#Data path

TYPE = 'NN'
PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'

DATA_PATH = PATH + 'DATA/'
PLOT_PATH = PATH + 'PLOT/' + TYPE + '/'

In [4]:
#Load in Data

BIN_SIZE = 4
GRID_SIZE = 128
DATA_SIZE = 100000

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as DATA:
    
    REDSHIFT_GRID = numpy.array(DATA['realizations/z'][:GRID_SIZE], dtype = 'float32')
    DATA_GRID = numpy.array(DATA['realizations/pdfs'][:DATA_SIZE, :BIN_SIZE, :GRID_SIZE], dtype = 'float32')

In [5]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as cosmo:
    
    COSMO = json.load(cosmo)

COSMO_CCL = pyccl.Cosmology(
    h = COSMO['H'],
    w0 = COSMO['W0'],
    wa = COSMO['WA'], 
    n_s = COSMO['NS'], 
    A_s = COSMO['AS'],
    m_nu = COSMO['MNU'],  
    Neff = COSMO['NEFF'],
    T_CMB = COSMO['TCMB'], 
    Omega_k = COSMO['OMEGAK'], 
    Omega_c = COSMO['OMEGAC'], 
    Omega_b = COSMO['OMEGAB'], 
    matter_power_spectrum = 'halofit',  
    transfer_function = 'boltzmann_camb', 
    extra_parameters = {'camb': {'kmax': 100, 'lmax': 10000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [6]:
#Restrcture the data

PSI_GRID = numpy.mean(DATA_GRID, axis = 0)
SIGMA_GRID = numpy.std(DATA_GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + REDSHIFT_GRID), dtype = 'float32')

CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + REDSHIFT_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 200
ELL_DATA = numpy.logspace(2, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE)) + numpy.inf, where = CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE))
for GRID_INDEX in range(GRID_SIZE):

    POWER_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [7]:
#Calculate the angular power spectra for different realizations
AMPLITUDE = 1 #3 / 2 * (COSMO['OMEGAM'] + COSMO['OMEGAN']) * (COSMO['H'] * 100000 / scipy.constants.c)**2 * numpy.sqrt((1 + 3 / (2 * ELL_DATA + 1)) * (1 + 1 / (2 * ELL_DATA + 1)) * (1 - 1 / (2 * ELL_DATA + 1)) * (1 - 3 / (2 * ELL_DATA + 1)))
T0 = time.time()
d_uv = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
DATA_SIZE_Cut = 2
DATA_GRID = list(DATA_GRID)
DATA_GRID=random.sample(DATA_GRID, DATA_SIZE_Cut)
PSI_GRID = numpy.mean(DATA_GRID, axis = 0)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + REDSHIFT_GRID)) * COSMO['H'] * 100000 / scipy.constants.c
PHI_GRID_cut = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, GRID_SIZE))
varepsilon_GRID_cut = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, GRID_SIZE))
C_uv_re = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
Bij=NNBij.function(amplitude = AMPLITUDE, chi_grid = CHI_GRID, power_grid = POWER_GRID, redshift_grid = REDSHIFT_GRID)
PHI_GRID_cut = DATA_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + REDSHIFT_GRID)) * COSMO['H'] * 100000 / scipy.constants.c
varepsilon_GRID_cut=PHI_GRID_cut-PHI_GRID
d_uv = NN.function(amplitude = AMPLITUDE, phi_grid = PHI_GRID, chi_grid = CHI_GRID, power_grid = POWER_GRID, redshift_grid = REDSHIFT_GRID)
T1 = time.time()
print(T1 - T0)

4.986715316772461


In [8]:
B_ij = numpy.zeros((DATA_SIZE_Cut,3, BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
T0 = time.time()
for a in range(DATA_SIZE_Cut):
    for m1 in range(BIN_SIZE):
        for m2 in range(BIN_SIZE):
            C_uv_re[a][m1][m2]=C_function(coefficients = Bij, phi_grid1 = PHI_GRID_cut[a][m1], phi_grid2 = PHI_GRID_cut[a][m2],grid_size=GRID_SIZE)
            B_ij[a][0][m1][m2]=C_function(coefficients = Bij, phi_grid1 = PHI_GRID[m1], phi_grid2 = PHI_GRID[m2],grid_size=GRID_SIZE)
            B_ij[a][1][m1][m2]=C_function(coefficients = Bij, phi_grid1 = PHI_GRID[m1], phi_grid2 = varepsilon_GRID_cut[a][m2],grid_size=GRID_SIZE)
            B_ij[a][2][m1][m2]=C_function(coefficients = Bij, phi_grid1 = varepsilon_GRID_cut[a][m1], phi_grid2 = varepsilon_GRID_cut[a][m2],grid_size=GRID_SIZE)
T1 = time.time()
print(T1 - T0)

0.465071439743042


In [9]:
#Calculate Gamma
Gamma_L = numpy.zeros((BIN_SIZE, BIN_SIZE, BIN_SIZE, BIN_SIZE, ELL_SIZE + 1, ELL_SIZE + 1))

for m1 in range(BIN_SIZE):
    for m2 in range(BIN_SIZE):
        for m1_prime in range(BIN_SIZE):
            for m2_prime in range(BIN_SIZE):
                for ell in range(ELL_SIZE + 1):
                    for ell_prime in range(ELL_SIZE + 1):
                        if m1 == m1_prime and m2 == m2_prime and ell == ell_prime:
                            Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime] = 1/(2*d_uv[m1][m2][ell]**2)

In [10]:
#Calculate third and Fourth order
Third_Fourth_order=numpy.zeros(DATA_SIZE_Cut)
nz = numpy.nonzero(Gamma_L)
T0 = time.time()
for a in range(DATA_SIZE_Cut):
    for idx in zip(*nz):
        m1, m2, m1_prime, m2_prime, ell, ell_prime = idx
        gamma_value = Gamma_L[idx]
        term1 = gamma_value * B_ij[a, 1, m2, m1, ell] * B_ij[a, 2, m1_prime, m2_prime, ell_prime]
        term2 = gamma_value * B_ij[a, 1, m1, m2, ell] * B_ij[a, 2, m1_prime, m2_prime, ell_prime]
        term3 = gamma_value * B_ij[a, 2, m1, m2, ell] * B_ij[a, 1, m2_prime, m1_prime, ell_prime]
        term4 = gamma_value * B_ij[a, 2, m1, m2, ell] * B_ij[a, 1, m1_prime, m2_prime, ell_prime]
        term5 = gamma_value * B_ij[a, 2, m1, m2, ell] * B_ij[a, 2, m1_prime, m2_prime, ell_prime]
        Third_Fourth_order[a] += term1 + term2 + term3 + term4 + term5
Third_Fourth_order *= -0.5
T1 = time.time()
print(T1 - T0)

0.03414416313171387


In [11]:
#Calculate All order
All_order=numpy.zeros(DATA_SIZE_Cut)
T0 = time.time()
for a in range(DATA_SIZE_Cut):
    for idx in zip(*nz):
        m1, m2, m1_prime, m2_prime, ell, ell_prime = idx
        gamma_value = Gamma_L[idx]
        All_order[a]=All_order[a]+(d_uv[m1][m2][ell]-C_uv_re[a][m1][m2][ell])*gamma_value*(d_uv[m1_prime][m2_prime][ell_prime]-C_uv_re[a][m1_prime][m2_prime][ell_prime])
All_order*= -0.5
First_Second_order=All_order-Third_Fourth_order
T1 = time.time()
print(T1 - T0)

0.0240633487701416


In [12]:
print(len(PHI_GRID))

4


In [13]:
#d_uv is d(l)
#PHI_GRID is Bar(Phi)
#Bij is Bij(l)
#varepsilon_GRID_cut is varepsilon
#Gamma_L is Gamma
#Define A(l)
#Define L_D
A_l = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
L_D = numpy.zeros(DATA_SIZE_Cut)
'''
for a in range(DATA_SIZE_Cut):
    for m1 in range(BIN_SIZE):
        for m2 in range(BIN_SIZE):
            A_l[a][m1][m2]=d_uv[m1][m2]=-C_function(coefficients = Bij, phi_grid1 = PHI_GRID_cut[a][m1], phi_grid2 = PHI_GRID_cut[a][m2],grid_size=GRID_SIZE)

for a in range(DATA_SIZE_Cut):
    for m1 in range(BIN_SIZE):
        for m2 in range(BIN_SIZE):
            for m1_prime in range(BIN_SIZE):
                for m2_prime in range(BIN_SIZE):
                    for ell in range(ELL_SIZE + 1):
                        for ell_prime in range(ELL_SIZE + 1):
                            L_D[a]= L_D[a]-0.5*A_l[a][m1][m2][ell]*Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime]*A_l[a][m1_prime][m2_prime][ell_prime]
'''

'\nfor a in range(DATA_SIZE_Cut):\n    for m1 in range(BIN_SIZE):\n        for m2 in range(BIN_SIZE):\n            A_l[a][m1][m2]=d_uv[m1][m2]=-C_function(coefficients = Bij, phi_grid1 = PHI_GRID_cut[a][m1], phi_grid2 = PHI_GRID_cut[a][m2],grid_size=GRID_SIZE)\n\nfor a in range(DATA_SIZE_Cut):\n    for m1 in range(BIN_SIZE):\n        for m2 in range(BIN_SIZE):\n            for m1_prime in range(BIN_SIZE):\n                for m2_prime in range(BIN_SIZE):\n                    for ell in range(ELL_SIZE + 1):\n                        for ell_prime in range(ELL_SIZE + 1):\n                            L_D[a]= L_D[a]-0.5*A_l[a][m1][m2][ell]*Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime]*A_l[a][m1_prime][m2_prime][ell_prime]\n'

In [ ]:
#Define b_mi
b_mi = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, GRID_SIZE))
Xi_mi = numpy.zeros((DATA_SIZE_Cut, BIN_SIZE, BIN_SIZE, GRID_SIZE, GRID_SIZE))
for m1 in range(BIN_SIZE):
    for m2 in range(BIN_SIZE):
        A_l[0][m1][m2]=d_uv[m1][m2]=-C_function(coefficients = Bij, phi_grid1 = PHI_GRID_cut[a][m1], phi_grid2 = PHI_GRID_cut[a][m2],grid_size=GRID_SIZE)
        for m1_prime in range(BIN_SIZE):
            for m2_prime in range(BIN_SIZE):
                for ell in range(ELL_SIZE + 1):
                    for ell_prime in range(ELL_SIZE + 1):
                        L_D[0]= L_D[0]-0.5*A_l[0][m1][m2][ell]*Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime]*A_l[0][m1_prime][m2_prime][ell_prime]
                        for i in range(GRID_SIZE):
                            for j in range(GRID_SIZE):
                                b_mi[0][m1][i]=b_mi[0][m1][i]-0.5*A_l[0][m1_prime][m2_prime][ell_prime]*Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime]*Bij[i][j][ell]*PHI_GRID[m2][j]\
                                -0.5*A_l[0][m1_prime][m2_prime][ell_prime]*Gamma_L[m2, m1, m1_prime, m2_prime, ell, ell_prime]*Bij[j][i][ell]*PHI_GRID[m2][j]\
                                -0.5*A_l[0][m1_prime][m2][ell]*Gamma_L[m1_prime, m2, m1, m2_prime, ell, ell_prime]*Bij[i][j][ell_prime]*PHI_GRID[m2_prime][j]\
                                -0.5*A_l[0][m2_prime][m2][ell]*Gamma_L[m2_prime, m2, m1_prime, m1, ell, ell_prime]*Bij[j][i][ell_prime]*PHI_GRID[m1_prime][j]
                                Xi_mi[0][m1][m2][i][j]=Xi_mi[0][m1][m2][i][j]-A_l[0][m1_prime][m2_prime][ell_prime]*(Gamma_L[m1, m2, m1_prime, m2_prime, ell, ell_prime]+Gamma_L[m1_prime, m2_prime, m1, m2, ell_prime, ell])*Bij[i][j][ell]
                                for i_prime in range(GRID_SIZE):
                                    for j_prime in range(GRID_SIZE):
                                        Xi_mi[0][m1][m2][i][j]=Xi_mi[0][m1][m2][i][j]+Gamma_L[m1, m1_prime, m2, m2_prime, ell, ell_prime]*Bij[i][i_prime][ell]*Bij[j][j_prime][ell_prime]*PHI_GRID[m1_prime][i_prime]*PHI_GRID[m2_prime][j_prime]\
                                         +Gamma_L[m1, m2_prime, m1_prime, m2, ell, ell_prime]*Bij[i][j_prime][ell]*Bij[i_prime][j][ell_prime]*PHI_GRID[m1_prime][i_prime]*PHI_GRID[m2_prime][j_prime]\
                                         +Gamma_L[m1_prime, m2, m1, m2_prime, ell, ell_prime]*Bij[i_prime][j][ell]*Bij[i][j_prime][ell_prime]*PHI_GRID[m1_prime][i_prime]*PHI_GRID[m2_prime][j_prime]\
                                         +Gamma_L[m2_prime, m2, m1_prime, m1, ell, ell_prime]*Bij[j_prime][j][ell]*Bij[i_prime][i][ell_prime]*PHI_GRID[m1_prime][i_prime]*PHI_GRID[m2_prime][j_prime]


In [ ]:
FS_order=L_D
for m1 in range(BIN_SIZE):
    for i in range(GRID_SIZE):
        FS_order[0]=FS_order[0]-b_mi[0][m1][i]*varepsilon_GRID_cut[0][m1][i]
for m1 in range(BIN_SIZE):
    for m2 in range(BIN_SIZE):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                FS_order[0]=FS_order[0]-0.5*Xi_mi[0][m1][m2][i][j]*varepsilon_GRID_cut[0][m1][i]*varepsilon_GRID_cut[0][m2][j]
print(First_Second_order)
print(FS_order)